In [1]:
import datetime
import os
import shutil
from pathlib import Path
from collections import Counter

import yaml
import numpy as np
import pandas as pd
# from ultralytics import YOLO
from sklearn.model_selection import KFold

# Generate dataset

In [2]:
dataset_path = Path('/media/jess/DATA/PhD/data/ecoflow/yolo_labels/14_classes/train/') # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*labels/*.txt")) # all data in 'labels'

In [3]:
yaml_file = '/media/jess/DATA/PhD/cnn_bakeoff2/yolo/config/trial_1.yaml'  # your data YAML with data directories and names dictionary
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)['names']
cls_idx = sorted(classes.keys())

In [4]:
indx = [l.stem for l in labels] # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

In [5]:
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    # labels_df.loc[label.stem] = lbl_counter
    labels_df.loc[label.stem] = [lbl_counter[cls] for cls in classes]

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`

In [6]:
labels_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
001A_2017_Hook_IMG_0016_028e1462,0,0,0,0,0,0,0,1,0,0,0,0,0,0
001A_2017_Hook_IMG_0436_efe2e308,0,0,0,0,0,0,0,1,0,0,0,0,0,0
001B_2017_Hook_IMG_0022_d0a6e9bc,0,0,0,0,0,0,0,1,0,0,0,0,0,0
001B_2017_Hook_IMG_0026_0b1f2d88,0,0,0,0,0,0,0,1,0,0,0,0,0,0
001B_2017_Hook_IMG_0123_225f268d,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TW_01_20180325_IMG_2157_66155066,0,0,0,0,0,0,1,0,0,0,0,0,0,0
TW_01_20180325_IMG_2160_96f64882,0,0,0,0,0,0,1,0,0,0,0,0,0,0
TW_02_20181019_IMG_0167_1fe4e6f9,0,0,0,0,0,0,1,0,0,0,0,0,0,0
TW_03_20171115_IMG_1112_ac8d3b89,0,0,0,0,0,0,0,0,1,0,0,0,0,0


# 5-fold data split

In [7]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [8]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

In [9]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

In [10]:
supported_extensions = ['.jpg', '.jpeg', '.png']

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / 'images').rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f'{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val')
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f'{split}_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        yaml.safe_dump({
            'path': split_dir.as_posix(),
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

In [11]:
for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

In [12]:
#save records
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

## Checking text files

In [13]:
def dataset_summary_b(path):
    
    classes_dict = {}

    for filename in os.listdir(path):
        if filename.endswith('.txt'):  # Ensure it's a text file
            with open(os.path.join(path, filename), 'r') as file:
                for line in file:
                    sp_class = line.split(' ')[0]
                    classes_dict[sp_class] = classes_dict.get(sp_class, 0) + 1

    return dict(sorted(classes_dict.items()))

In [14]:
print(dataset_summary_b('/media/jess/DATA/PhD/data/ecoflow/yolo_labels/14_classes/train/2024-05-09_5-Fold_Cross-val/split_1/train/labels'))
print(dataset_summary_b('/media/jess/DATA/PhD/data/ecoflow/yolo_labels/14_classes/train/2024-05-09_5-Fold_Cross-val/split_2/train/labels'))
print(dataset_summary_b('/media/jess/DATA/PhD/data/ecoflow/yolo_labels/14_classes/train/2024-05-09_5-Fold_Cross-val/split_3/train/labels'))
print(dataset_summary_b('/media/jess/DATA/PhD/data/ecoflow/yolo_labels/14_classes/train/2024-05-09_5-Fold_Cross-val/split_4/train/labels'))
print(dataset_summary_b('/media/jess/DATA/PhD/data/ecoflow/yolo_labels/14_classes/train/2024-05-09_5-Fold_Cross-val/split_5/train/labels'))

{'0': 807, '1': 787, '10': 791, '11': 798, '12': 809, '13': 780, '2': 810, '3': 816, '4': 793, '5': 788, '6': 805, '7': 793, '8': 801, '9': 822}
{'0': 801, '1': 811, '10': 818, '11': 802, '12': 796, '13': 807, '2': 794, '3': 795, '4': 800, '5': 806, '6': 791, '7': 813, '8': 772, '9': 794}
{'0': 815, '1': 784, '10': 792, '11': 797, '12': 803, '13': 796, '2': 818, '3': 797, '4': 807, '5': 797, '6': 794, '7': 792, '8': 804, '9': 804}
{'0': 809, '1': 804, '10': 803, '11': 805, '12': 780, '13': 808, '2': 799, '3': 793, '4': 786, '5': 819, '6': 807, '7': 786, '8': 807, '9': 794}
{'0': 768, '1': 814, '10': 796, '11': 798, '12': 812, '13': 809, '2': 779, '3': 799, '4': 814, '5': 790, '6': 803, '7': 816, '8': 816, '9': 786}


# Run YOLO

In [1]:
#change to yolov8 env
import torch
from ultralytics import YOLO

device = '0' if torch.cuda.is_available() else 'cpu'
if device=='0':
    torch.cuda.set_device(0)
print(f'Device: {device}')

model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')
model = YOLO('yolov8n.yaml').load('yolov8n.pt')

Device: 0
Transferred 355/355 items from pretrained weights


In [15]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

ksplit = 5
results = {}

# Define your additional arguments here
batch = 64
project = 'kfold_1000images'
epochs = 100
patience = 10

for k in range(ksplit):
    ds_yaml = f'/media/jess/DATA/PhD/data/ecoflow/yolo_labels/14_classes/train/2024-05-09_5-Fold_Cross-val/split_{k+1}/split_{k+1}_dataset.yaml'
    dataset_yaml = ds_yaml
    model.train(data=dataset_yaml,
                epochs=epochs,
                batch=batch,
                patience=patience,
                project=project)  # include any train arguments
    results[k+1] = model.metrics  # save output metrics for further analysis

print("Training results:", results)

Ultralytics YOLOv8.2.11 🚀 Python-3.10.13 torch-2.3.0+cu121 CUDA:0 (NVIDIA RTX A6000, 48669MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/media/jess/DATA/PhD/data/ecoflow/yolo_labels/14_classes/train/2024-05-09_5-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=100, time=None, patience=10, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=kfold_1000images, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jesstytam. Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


/home/jess/mambaforge/envs/yolov8/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:10<00:00,  2.08it/s]


                   all       2800       2800      0.982      0.989      0.993      0.932
       Brown Bandicoot       2800        232      0.923       0.98      0.978       0.83
    Red-necked Wallaby       2800        186      0.982      0.995      0.995       0.96
      Brushtail Possum       2800        221      0.996          1      0.995      0.969
                   Cat       2800        201      0.994       0.99      0.995      0.923
               Red Fox       2800        186      0.994      0.995      0.995       0.96
           Rabbit Hare       2800        210      0.982      0.986      0.995      0.907
                   Dog       2800        197      0.992       0.99      0.995      0.945
 Eastern Grey Kangaroo       2800        184      0.976      0.995      0.995      0.955
               Echidna       2800        184      0.994      0.982      0.995      0.915
                   Pig       2800        214      0.986          1      0.995      0.956
                  Eur

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


lr/pg0,▁▅█████▇▇▇▇
lr/pg1,▁▅█████▇▇▇▇
lr/pg2,▁▅█████▇▇▇▇
metrics/mAP50(B),█▇▅▁▃▂▆▃▃▆█
metrics/mAP50-95(B),█▇▅▁▂▂▅▃▂▅█
metrics/precision(B),█▇▄▁▃▂▅▃▃▆█
metrics/recall(B),█▇▄▁▃▁▅▂▂▅█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▁▂▄▇████▇▇▇


Training results: {1: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f265bb40d30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.04304